##### You are now ready to use Python and SQLAlchemy to do basic climate analysis and data exploration on your new weather station tables. All of the following analysis should be completed using SQLAlchemy ORM queries, Pandas, and Matplotlib.
-  Choose a start date & end date for your trip (make sure range is approx. 3-15 days total.
-  Use SQLAlchemy 'create_engine' to connect to sqlite database.
-  Use SQLAlchemy 'automap_base()' to reflect your tables into classes and save a reference to those classes called 'Station' and 'Measurement'.

In [56]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData, Table
from sqlalchemy import Column, Date, Float, Integer, String

In [57]:
# 6/11/2017 ~ 6/18/2017
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

In [58]:
metadata = MetaData()
#print metadata.tables

metadata.reflect(bind=engine)
for table in metadata.tables.values():
    print(table.name)
    for column in table.c:
        print(column.name)

measurements
id
station
date
prcp
tobs
stations
id
station
name
latitude
longitude
elevation


In [59]:
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'stations']

In [60]:
engine.execute('Select * from measurements LIMIT 5').fetchall()

[(0, 'USC00519397', '1/1/2010', 0.08, 65),
 (1, 'USC00519397', '1/2/2010', 0.0, 63),
 (2, 'USC00519397', '1/3/2010', 0.0, 74),
 (3, 'USC00519397', '1/4/2010', 0.0, 76),
 (4, 'USC00519397', '1/7/2010', 0.06, 70)]

In [61]:
#def addKeys(TableName):
#    class AddPrimaryKey(Base):
#        __tablename__ = TableName
#        __table_args__ = {"extend_existing" : True}
#        id = Column(Integer, primary_key = True)
#        station = Column(String, primary_key = True)

#addKeys('measurements')
#addKeys('stations')


In [62]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

[]

In [55]:
session.query(func.count(Meas.date)).all()

NameError: name 'Meas' is not defined

[]

### Precipitation Analysis
-  Design a query to retrieve the last 12 months of precipitation data.

-  Select only the `date` and `prcp` values.

-  Load the query results into a Pandas DataFrame and set the index to the date column.

-  Plot the results using the DataFrame `plot` method.

    -  <center><img src='images/precip.png' /></center>

-  Use Pandas to print the summary statistics for the precipitation data.

### Station Analysis

-  Design a query to calculate the total number of stations.

-  Design a query to find the most active stations.

    -  List the stations and observation counts in descending order

    -  Which station has the highest number of observations?

-  Design a query to retrieve the last 12 months of temperature observation data (tobs).

    -  Filter by the station with the highest number of observations.

    -  Plot the results as a histogram with `bins=12`.

  <center><img src='images/temp_hist.png' height="400px" /></center>

### Temperature Analysis

-  Write a function called `calc_temps` that will accept a start date and end date in the format `%Y-%m-%d` and return the minimum, average, and maximum temperatures for that range of dates.

-  Use the `calc_temps` function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")

-  Plot the min, avg, and max temperature from your previous query as a bar chart.

    -  Use the average temperature as the bar height.

    -  Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

<center><img src='images/temp_avg.png' height="30px"/></center>